Based on **Francesco Pierfederici: Distributed Computing with Python, Chapter 2**

# Iterators

Most Python programmers are familiar with the concept of **iterating some sort of 
collection** (for example, strings, lists, tuples, ranges, file objects, and so on):


In [1]:
type(range(3))

range

Iterating over a range:

In [2]:
for i in range(3):
    print(i)

0
1
2


Iterating over the lines of a text file:

In [3]:
for line in open('data/hello_world.txt'):
...     print(line, end='')

Hello
World!
How are you doing today?
:-)




The reason why we can iterate all sorts of objects and not just lists or strings is the 
**iteration protocol**. 


The iteration protocol defines a standard interface for iteration: 
an object that implements **\__iter__** and **\__next__** (or \__iter__ and next in Python 
2.x) is an **iterator** and, as the name suggests, can be iterated over, as shown in the 
following code snippet

Sequences ( list, tuple, range), always support the iteration methods.


In [4]:
range(3).__iter__

<method-wrapper '__iter__' of range object at 0x7eff0c607a80>

In [5]:
['a','b','c'].__iter__

<method-wrapper '__iter__' of list object at 0x7eff0c60fbc8>

In [6]:
tmpfile=open('data/hello_world.txt')
tmpfile.__next__
tmpfile.__iter__

<method-wrapper '__iter__' of _io.TextIOWrapper object at 0x7eff0c6022d0>

We can define the \__iter__ and \__next__ methods for our classes:

In [7]:
class MyIterator(object):
    def __init__(self, xs):
        self.xs = xs

    def __iter__(self):
        return self

    def __next__(self):
        if self.xs:
            return self.xs.pop(0)
        else:
            raise StopIteration

In [8]:
for i in MyIterator(['a', 'b', 'c']):
    print(i)

a
b
c


**\__iter__**   returns the object we iterate, and the **\__next__** method returns the individual 
elements of the sequence one by one.

To better see how the protocol works, we can unroll the loop manually as the 
following piece of code shows:


In [9]:
itrtr = MyIterator([3, 4, 5, 6])
print(itrtr)

In [10]:
print(next(itrtr))
print(next(itrtr))

3
4


Once the sequence is exhausted, next() throws a **StopIteration exception**. 

The for loop in Python, for instance, uses the same mechanism; it calls 
next() on its iterator and catches the StopIteration exception to know when to stop.

In [11]:
print(next(itrtr))
print(next(itrtr))
print(next(itrtr))

5
6


StopIteration: 

# Generators
Generators can also be used to iterate over objects.

A generator is simply a callable function that **generates a sequence of results rather than 
returning a result**. This is achieved by **yielding** (by way of using the yield keyword) 
the individual values **rather then returning them**, as we can see in the following 
example:


In [12]:
def mygenerator(n):
    while n:
        n -= 1
        yield n # we use yield instead of return

Example of using the mygenerator generator:

In [13]:
for i in mygenerator(3):
    print(i)

2
1
0


It is the simple **presence of yield that makes mygenerator a generator and not a 
simple function**. 

The interesting behavior in the preceding code is that calling the 
generator function **does not start the generation of the sequence** at all; it just creates 
a generator object, as the following example shows:

In [14]:
g=mygenerator(2)

In [15]:
next(g)

1

In [16]:
next(g)

0

In [17]:
next(g)

StopIteration: 

Each **next()** call produces a value from the generated sequence until the sequence 
is empty, and that is when we get the **StopIteration exception** instead. 

This is the same behavior that we saw when we looked at iterators. 

Essentially, generators  are a **simple way to write iterators without the need for defining classes with their 
\__iter__ and \__next__ methods.**

As a side note, you should keep in mind that generators are one-shot operations; it is 
not possible to iterate the generated sequence more than once. To do that, you have 
to call the generator function again.

# Coroutines 

The same yield expression used in generator functions to produce a sequence of values **can be used on the right-hand side of an assignment** to consume values. 

This allows the creation of **coroutines**.

A coroutine is simply a type of function that can **suspend and resume its execution at well-defined locations** in its code  via **yield** expressions.


### Let's create some coroutines and see how we can use them. 

There are three main  constructs in coroutines, which are stated as follows:

• **yield()**: This is used to **suspend the execution** of the coroutine <br>
• **send()**: This is used to **pass data to a coroutine** (and hence **resume** its execution) <br>
• **close()**: This is used to **terminate** a coroutine <br>

In [18]:
def complain_about(substring):
    print('Please talk to me!')
    try:
        while True:
            text = (yield)
            if substring in text:
                print('Oh no: I found a %s again!' % (substring))
    except GeneratorExit:
        print('Ok, ok: I am quitting.')

We start off by defining our coroutine; it is just a function (we called it complain_about) that takes a single argument: a string. 

After printing a message, it **enters an infinite loop** enclosed in a try except clause. 

This means that the only way to exit the loop is via an exception. 

We are particularly interested in a very specific exception: **GeneratorExit**. When we catch one of these, we simply clean up and quit.

The body of the loop itself is pretty simple; **we use a yield expression to fetch data** (using the (yield) expression) and store it in the variable "text". Then, we simply check whether the "substring" variable is in the "text" variable, and if so, we whine a bit.

In [19]:
c = complain_about('Machine Learning')

We need to start the c corutine with the **next(c)** command.It will run the "complain_about" function **till the first (yield)** command.

The execution of complain_about('Machine Learning') **creates the coroutine, but nothing else 
seems to happen**. 

In order to use the newly created coroutine, **we need to call next() 
on it**, just like we had to do with generators. 

In fact, we see that it is only after calling next() that we get "Please talk to me!" printed on the screen.

At this point, the coroutine has reached the text = (yield) line, which means that it **suspends its execution**. 

The **control goes back to the interpreter** so that we can send data to the coroutine itself. 

We do that using the its **send()** method.

In [20]:
next(c) 

Please talk to me!


In [21]:
c.send('Test data')

In [22]:
c.send('Some more random text')

In [23]:
c.send('I love Machine Learning because it is so cool!')

Oh no: I found a Machine Learning again!


In [24]:
c.send('Hello')

We can use the "next(c)" command **only once**:

In [25]:
next(c)

TypeError: argument of type 'NoneType' is not iterable

We can stop the coroutine by calling its **close()** method, which results in a 
**GeneratorExit exception** being risen inside the coroutine. 

The only thing that a coroutine is allowed to do at this point is catch the exception, do some cleaning up, 
and exit. The following snippet shows how to **close the coroutine**:

In [26]:
c.close()

### Decorators

When using coroutines, most people find **having to call next() on the coroutine 
rather annoying** and end up **using a decorator to avoid the extra call**, as the following 
example shows:

In [27]:
# Let us create a wrapper that will always call the next(c) function for us.
def coroutine(fn):
    # We will apply the coroutin function to the fn function.
    # inside the coroutine function we create another function called wrapper
    def wrapper(*args, **kwargs):
        c = fn(*args, **kwargs)
        next(c)
        return c
    return wrapper

Example of using the decorator:

In [28]:
@coroutine
def complain_about2(substring):
    print('Please talk to me!')
    while True:
        text = (yield)
        if substring in text:
            print('Oh no: I found a %s again!'% (substring))
            

Now we can run this coroutine without starting it with the next statement:

In [29]:
c = complain_about2('JavaScript')

Please talk to me!


In [30]:
c.send('Test data with JavaScript somewhere in it')

Oh no: I found a JavaScript again!


In [31]:
c.send('Hello')

In [32]:
c.close()

Coroutines can be arranged in rather **complex hierarchies**, with one **coroutine sending 
data to multiple other ones** and **getting data from multiple sources** as well

Since coroutines **can stop there execution** and **wait in a non-blocking wait** till we restart them again, they can be used in **asynchronous programming**:

# An asynchronous example

To keep things simple but still interesting, let's write a tool that, given a text file, will 
count the occurrences of a given word

In [33]:
# Run the below line in a terminal:
# time(grep -io love data/pg2600.txt | wc -l)
#
# This will count the number of "love" work in the data/pg2600.txt file
# It will also measure the executation time.
# Results:


677

real    0m0.014s <br>
user    0m0.008s <br>
sys     0m0.004s <br>


We will solve the word counting problem with coroutines 

In [34]:
def coroutine(fn):
    def wrapper(*args, **kwargs):
        c = fn(*args, **kwargs)
        next(c)
        return c
    return wrapper

In [35]:
#Reading the file line by line (done by the cat function)

#The first function, cat, acts as the data source for the whole program; it reads the 
#file line by line and sends each line to its child function via child.send(line). 
# This child function will be the grep() function

#If we want a case-insensitive match, then we simply make line lowercase; otherwise, we pass it unchanged.

def cat(f, case_insensitive, child):
    if case_insensitive:
        line_processor = lambda l: l.lower()
    else:
        line_processor = lambda l: l

    for line in f:
        child.send(line_processor(line))

In [36]:
#Counting the occurrences of substring in each line

#The grep function is our first coroutine. 

#In it, we enter an infinite loop where we keep receiving data (text = (yield)), 
#count the occurrences of the "substring" variable in the "text" variable, 
#and send that number of occurrences to the next coroutine (count in our case)
# to count how many times we have seen the "substring" variable.

#child.send(text.count(substring))).

@coroutine
def grep(substring, case_insensitive, child):
    if case_insensitive:
        substring = substring.lower()
    while True:
        text = (yield)
        child.send(text.count(substring))

In [37]:
#Adding up all the numbers and printing out the total

#The count coroutine keeps a running total, n, of the numbers it receives, 
#(n += (yield)), from grep. 

#It catches the GeneratorExit exception sent to each coroutine 
#when they are closed (which in our case happens automatically when we reach the 
#end of the file) to know when to print out substring and n

@coroutine
def count(substring):
    n = 0
    try:
        while True:
            n += (yield)
    except GeneratorExit:
        print(substring, n)

This is how we can use these corutines:

In [38]:
cat(f=open('data/pg2600.txt'), case_insensitive=True, child=grep(substring='love', case_insensitive=True, child=count('love')))

love 677


Things become interesting when we start organizing coroutines into complex  graphs. 

For instance, we might want to count the concurrence of multiple words in the input file.

The below code is an example for this:

In [39]:
def coroutine(fn):
    def wrapper(*args, **kwargs):
        c = fn(*args, **kwargs)
        next(c)
        return c
    return wrapper

def cat(f, case_insensitive, child):
    if case_insensitive:
        line_processor = lambda l: l.lower()
    else:
        line_processor = lambda l: l

    for line in f:
        child.send(line_processor(line))

@coroutine
def grep(substring, case_insensitive, child):
    if case_insensitive:
        substring = substring.lower()
    while True:
        text = (yield)
        child.send(text.count(substring))

@coroutine
def count(substring):
    n = 0
    try:
        while True:
            n += (yield)
    except GeneratorExit:
        print(substring, n)

@coroutine
def fanout(children):
    while True:
        data = (yield)
        for child in children:
            child.send(data)

In [40]:
cat(f=open('data/pg2600.txt'), case_insensitive=True, child=fanout(children=[grep(substring=p,case_insensitive=True,child=count(p)) for p in ['love','hate','hope']]))

hate 103
love 677
hope 158


### Other tools:
Python 3.4 introduced a new library for asynchronous I/O called asyncio.

Python 3.5 introduced true coroutine types via async def and await.

We will discuss them later.